In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, evaluate, LinearTransform
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using LinearMaps

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


In [3]:
Nx = 80
Ne = 300
X = zeros(randn(Nx, Ne))
X0  = deepcopy(ens.S)
Xin = deepcopy(ens.S)
Xout = deepcopy(Xin)

cov(ens)
σ = std(ens.S; dims = 2)[:,1];

In [4]:
L = LinearTransform(Xin; diag = false)
lmap = cholesky(cov(X0')).L
invlmap = inv(lmap)

80×80 LowerTriangular{Float64,Array{Float64,2}}:
  0.901554      ⋅             ⋅           …    ⋅          ⋅          ⋅ 
 -0.0697431    1.01013        ⋅                ⋅          ⋅          ⋅ 
  0.0620572   -0.0401903     0.94172           ⋅          ⋅          ⋅ 
 -0.00651134  -0.116109     -0.0515584         ⋅          ⋅          ⋅ 
  0.0115551   -0.0279193     0.0952991         ⋅          ⋅          ⋅ 
 -0.0225679   -0.0359659    -0.055617     …    ⋅          ⋅          ⋅ 
  0.0303757   -0.00828113    0.104551          ⋅          ⋅          ⋅ 
 -0.011057    -0.00730934    0.0155775         ⋅          ⋅          ⋅ 
 -0.00805503  -0.0755582    -0.002607          ⋅          ⋅          ⋅ 
 -0.00533862  -0.0338618    -0.1212            ⋅          ⋅          ⋅ 
  0.113411    -0.0565354     0.0950428    …    ⋅          ⋅          ⋅ 
  0.0474702   -0.0408741     0.0364002         ⋅          ⋅          ⋅ 
  0.0486044   -0.0728819     0.0248711         ⋅          ⋅          ⋅ 
  ⋮            

Test Summary:                                | Pass  Total
Linear rescaling of multi-dimensional sample |    6      6


Test.DefaultTestSet("Linear rescaling of multi-dimensional sample", Any[], 6, false)

In [82]:
?transpose

search: transpose transpose! Transpose copy_transpose! TransportMap



```
transpose(A)
```

Lazy transpose. Mutating the returned object should appropriately mutate `A`. Often, but not always, yields `Transpose(A)`, where `Transpose` is a lazy transpose wrapper. Note that this operation is recursive.

This operation is intended for linear algebra usage - for general data manipulation see [`permutedims`](@ref Base.permutedims), which is non-recursive.

# Examples

```jldoctest
julia> A = [3+2im 9+2im; 8+7im  4+6im]
2×2 Array{Complex{Int64},2}:
 3+2im  9+2im
 8+7im  4+6im

julia> transpose(A)
2×2 Transpose{Complex{Int64},Array{Complex{Int64},2}}:
 3+2im  8+7im
 9+2im  4+6im
```


In [73]:
X = 2.0 .+ randn(Nx, Ne) .* randn(Nx, Ne)

1×100 Array{Float64,2}:
 2.19043  2.33579  2.66839  1.98909  …  1.10198  2.71694  5.96888  2.16165

In [78]:
(X .- mean(X; dims = 2))*(X .- mean(X; dims = 2))'

1×1 Array{Float64,2}:
 76.70818229198677

In [74]:
cov(X')

1×1 Array{Float64,2}:
 0.7748301241614827

In [68]:
randn(Nx, Ne) .* randn(Nx, Ne)

1×100 Array{Float64,2}:
 1.90006  -0.0833448  -0.0686482  …  -0.00709509  0.0811489  2.46677

In [67]:
randn(Nx,Ne)

1×100 Array{Float64,2}:
 0.729149  -0.796721  1.98627  1.85363  …  -1.22494  -1.57532  1.55905

In [53]:
@time transform!(L, Xout, Xin)

  0.000161 seconds


80×300 Array{Float64,2}:
  1.04677      0.508415   0.971975   …  -0.571998     1.2209    -0.23855
  0.430134    -0.190924  -1.9813         2.19622     -0.604119  -0.697181
  1.02528      1.71991   -1.53037       -0.0778346   -0.425632  -0.228743
  0.209198    -0.543831   0.0356501     -0.584034    -1.33618   -0.984829
  0.74486      0.417286   0.4307         0.798131    -1.66195   -0.154572
  1.5594       1.193      0.955936   …  -1.52349      0.254767  -1.19313
  0.257142    -1.2768     0.706037      -0.836838     0.582857  -0.306916
 -0.00130528  -0.538837   0.617927      -0.952581    -1.08792   -0.507733
 -0.686832     0.352179   0.338563       0.779768    -0.681399  -0.650478
 -0.263848     0.443115   0.435429      -0.522251    -1.03145   -0.366854
  0.135462     0.680577   0.651573   …   1.16669     -0.708437   1.0513
 -0.126137     1.56389   -0.270601       0.0220606   -0.446768  -1.46059
  0.656142     0.532647  -1.55132        1.72033      0.702405  -0.657909
  ⋮               

In [37]:
cov(Xout')

80×80 Array{Float64,2}:
  1.0           1.33672e-17   8.91149e-18  …  -1.7823e-17    6.68362e-18
  1.33672e-17   1.0           8.91149e-17     -3.49033e-17  -9.65411e-17
  8.91149e-18   8.91149e-17   1.0             -5.94099e-18   5.94099e-18
  1.7823e-17    1.1882e-17   -8.16887e-18     -1.1882e-17    2.9705e-17
 -1.7823e-17   -1.85656e-17   0.0              4.75279e-17  -4.75279e-17
  3.5646e-17   -2.3764e-17   -1.1882e-17   …  -5.04984e-17  -5.19837e-17
  1.1882e-17    5.94099e-17  -1.1882e-17       5.94099e-17  -1.87141e-16
 -2.36711e-17   4.13085e-18  -5.94099e-18      4.75279e-17   2.9705e-17
  1.1882e-17    1.7823e-17   -5.64394e-17      1.48525e-17   5.34689e-17
  2.3764e-17   -8.31739e-17  -2.82197e-17     -5.34689e-17  -5.04984e-17
  1.48525e-17   8.31739e-17  -5.94099e-17  …  -4.75279e-17   6.83214e-17
  3.5646e-17   -4.75279e-17   1.93082e-17      1.48525e-17  -3.5646e-17
  5.19837e-17  -2.07935e-17   3.26755e-17      1.24761e-16   1.30702e-16
  ⋮                           

In [33]:
invlmap * X0[:,1:2]

80×2 Array{Float64,2}:
  0.0491569   0.239869
 -0.296063   -1.15788
 -0.121559    0.334596
 -0.155733   -0.29154
  1.26689     0.284457
  1.71508     0.0734081
  0.0218186   0.698705
 -1.03074    -0.234905
 -0.0828186  -0.0822016
 -1.08167    -0.215047
  0.653063   -0.711922
  0.113864    0.231887
  0.10172     1.47721
  ⋮          
  0.582942    0.133925
  0.545938    0.466682
 -0.334806   -1.24085
 -1.01459    -0.137535
 -0.816284   -0.331453
  0.997885   -1.19477
 -0.660938    0.282105
 -2.21467    -0.276572
  1.2815     -1.60829
  1.56761     0.147073
 -1.14959    -0.350411
 -0.488614   -2.06581

In [15]:
@time transform!(L, Xout)

  0.000181 seconds


80×300 Array{Float64,2}:
 -1.3351    -0.569898    -0.677604  …  -1.59083     0.345324    0.410673
  0.036491  -0.409071     0.502743      1.26056    -1.25023     0.805386
  1.1676     0.823829    -0.763641     -0.0869958   0.272985    1.86174
  1.09866   -1.82105      0.54774       0.273295   -1.61182     0.284582
 -0.655113   0.379747    -1.39377      -0.61716     0.0782533  -0.43047
 -0.252955  -0.684055     0.237642  …   0.217708   -0.965699   -0.33376
  0.160546  -0.334613     0.955562      1.46613    -1.26213    -0.225261
  0.124025   0.722054     0.722715      1.47763     1.15348    -0.453709
  0.984261   1.12964     -0.865224     -0.991418   -1.72183    -0.109204
 -1.33522    0.17543      2.56723       0.701497   -0.997316   -0.472932
  2.20739   -0.482064    -0.271813  …   0.532369   -0.441161   -0.829815
 -1.21684   -0.0588271    0.915605      1.92973     1.09845    -0.854066
  0.369048   0.658913    -0.823891     -0.120967   -0.86439    -0.512733
  ⋮                          

In [22]:
inv(cholesky(cov(X0')).L)*cholesky(cov(X0')).L

80×80 LowerTriangular{Float64,Array{Float64,2}}:
  1.0            ⋅           …    ⋅            ⋅            ⋅    ⋅ 
  6.87414e-19   1.0               ⋅            ⋅            ⋅    ⋅ 
  5.56735e-18  -1.90445e-18       ⋅            ⋅            ⋅    ⋅ 
 -1.33153e-17  -1.61681e-18       ⋅            ⋅            ⋅    ⋅ 
 -7.58213e-18  -1.11743e-19       ⋅            ⋅            ⋅    ⋅ 
 -1.07718e-17   2.66256e-18  …    ⋅            ⋅            ⋅    ⋅ 
 -8.05545e-18  -1.36033e-17       ⋅            ⋅            ⋅    ⋅ 
  4.41038e-18   1.4615e-17        ⋅            ⋅            ⋅    ⋅ 
 -1.08475e-17   7.06431e-19       ⋅            ⋅            ⋅    ⋅ 
 -6.11817e-18   3.5401e-17        ⋅            ⋅            ⋅    ⋅ 
  7.37963e-18  -5.87328e-19  …    ⋅            ⋅            ⋅    ⋅ 
 -1.57127e-17  -2.21904e-18       ⋅            ⋅            ⋅    ⋅ 
 -7.59569e-18   1.57448e-17       ⋅            ⋅            ⋅    ⋅ 
  ⋮                          ⋱                                  
  

In [16]:
cov(Xout')

80×80 Array{Float64,2}:
  1.0           1.1882e-17   -5.04984e-17  …   4.75279e-17  -2.9705e-17
  1.1882e-17    1.0           1.1882e-17      -1.7823e-17    0.0
 -5.04984e-17   1.1882e-17    1.0              2.3764e-17   -3.86165e-17
 -1.48525e-17  -8.91149e-18  -6.23804e-17     -1.28103e-16  -4.15869e-17
  4.45574e-17   5.94099e-18  -1.03967e-17      7.42624e-18   3.5646e-17
  6.79501e-17  -5.94099e-18  -6.53509e-17  …   2.67345e-17   2.07935e-17
  7.16632e-17  -1.1882e-16   -2.3764e-17       3.11902e-17   3.26755e-17
 -2.9705e-18   -3.26755e-17   8.91149e-18     -1.03967e-17   9.20854e-17
  3.86165e-17   2.67345e-17  -6.68362e-18     -1.0768e-17    2.9705e-18
  2.3764e-17   -7.12919e-17   6.98067e-17      8.31739e-17  -8.91149e-18
  1.7823e-17    1.1882e-17    2.9705e-17   …   4.27009e-18  -2.3764e-17
  5.86673e-17  -5.94099e-18  -5.19837e-18      3.5646e-17    2.9705e-17
 -5.94099e-17  -1.93082e-17   1.7823e-17       1.25503e-16   4.60427e-17
  ⋮                                     

In [8]:
fieldnames(typeof(L.L)).lmap

(:lmap, :_issymmetric, :_ishermitian, :_isposdef)

In [9]:
L.L.lmap

80×80 LowerTriangular{Float64,Array{Float64,2}}:
  0.945581       ⋅             ⋅           …    ⋅           ⋅           ⋅ 
 -0.0239806     0.978408       ⋅                ⋅           ⋅           ⋅ 
 -0.0174472     0.00506721    0.980191          ⋅           ⋅           ⋅ 
 -0.0321765     0.0596801     0.0349768         ⋅           ⋅           ⋅ 
 -0.061504      0.0311873     0.0309401         ⋅           ⋅           ⋅ 
  0.143545     -0.0211021     0.0612896    …    ⋅           ⋅           ⋅ 
 -0.0169812    -0.0232876     0.0807928         ⋅           ⋅           ⋅ 
  6.27264e-5   -0.0698455    -0.065936          ⋅           ⋅           ⋅ 
 -0.000823668   0.0241513    -0.000524729       ⋅           ⋅           ⋅ 
 -0.0619665    -0.0271935    -0.0424068         ⋅           ⋅           ⋅ 
  0.0474302     0.0264961     0.042484     …    ⋅           ⋅           ⋅ 
  0.0315693    -0.0780691     0.120115          ⋅           ⋅           ⋅ 
 -0.0985533    -0.0236438     0.071179          ⋅  

In [47]:
L.L*(Xin .- mean(Xin; dims = 2)[:,1])

LinearMaps.CompositeMap{Float64,Tuple{LinearMaps.WrappedMap{Float64,Array{Float64,2}},LinearMaps.WrappedMap{Float64,LowerTriangular{Float64,Array{Float64,2}}}}}((LinearMaps.WrappedMap{Float64,Array{Float64,2}}([-1.6963735602396064 -1.1681321805751932 … 1.9655925386541142 0.1452532970572898; 0.8674320373585482 -0.07958058245849785 … 1.4815270658250943 -1.764044412347645; … ; -0.341438335879467 0.48148915801452874 … 0.09129948032192876 -1.9834540463777688; -0.99402247337154 -1.5148181044864943 … 0.2675510809915622 0.05138907087223119], false, false, false), LinearMaps.WrappedMap{Float64,LowerTriangular{Float64,Array{Float64,2}}}([0.9092317807000679 0.0 … 0.0 0.0; -0.042021016544533205 1.033031862353308 … 0.0 0.0; … ; -0.03354780288036603 -0.03413152292777033 … 1.1723220697582977 0.0; -0.13133582147679823 0.06431061236909699 … -0.1078144109766392 1.2055872065007853], false, false, false)))

In [55]:
Xout = deepcopy(Xin)
Xout .-= mean(Xin; dims = 2)[:,1]

80×300 Array{Float64,2}:
 -0.236096   -0.251663   -1.14241    …  -0.602429    0.738946    1.40692
  0.765474    0.971487    1.52985        1.02665    -0.887172    1.37213
  0.219347    0.454565   -0.700333       1.41106     0.179832    1.43021
 -0.779415    0.327365    1.25451       -0.893321   -0.449719   -0.0239816
 -0.989809    0.226404   -0.160728      -0.995774    1.1215      0.321248
  0.266979    0.576621    0.147646   …   1.35671    -0.42086     1.0479
  0.412401   -0.93396    -0.323714       1.0714      2.12364     1.41126
  3.35116     0.0613978  -0.0304457      0.634996    0.109942    1.10835
 -0.20152    -0.981175   -1.64355       -0.606314   -1.19392     0.772398
 -0.640636    1.63116    -1.49769       -0.826209   -1.85778    -1.80562
 -0.794126    1.74006    -0.729381   …  -0.668818   -0.0585668   0.983216
 -0.836031    0.0910632  -0.681829       1.51336     0.461504    0.710838
  0.73963    -0.366545    0.435172      -1.99555     0.0737865  -1.69808
  ⋮                  

In [56]:
for i=1:300
    mul!(Xout[:,i], L.L, Xout[:,i])
end

In [58]:
?ldiv!

search: ldiv! splitdrive



┌ Error: Failed to revise /media/mat/HDD/AdaptiveTransportMap/src/tools/scale.jl
│   exception = (LoadError("/media/mat/HDD/AdaptiveTransportMap/src/tools/scale.jl", 66, "incomplete: \"function\" at /media/mat/HDD/AdaptiveTransportMap/src/tools/scale.jl:31 requires end"), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007f36a0310193])
└ @ Revise /home/mat/.julia/packages/Revise/BqeJF/src/Revise.jl:760
┌ Warning: Due to a previously reported error, the running code does not match saved version for the following files:
│ 
│   /media/mat/HDD/AdaptiveTransportMap/src/tools/scale.jl
│ 
│ Use Revise.errors() to report errors again.
└ @ Revise /home/mat/.julia/packages/Revise/BqeJF/src/Revise.jl:820


```
ldiv!(Y, A, B) -> Y
```

Compute `A \ B` in-place and store the result in `Y`, returning the result.

The argument `A` should *not* be a matrix.  Rather, instead of matrices it should be a factorization object (e.g. produced by [`factorize`](@ref) or [`cholesky`](@ref)). The reason for this is that factorization itself is both expensive and typically allocates memory (although it can also be done in-place via, e.g., [`lu!`](@ref)), and performance-critical situations requiring `ldiv!` usually also require fine-grained control over the factorization of `A`.

# Examples

```jldoctest
julia> A = [1 2.2 4; 3.1 0.2 3; 4 1 2];

julia> X = [1; 2.5; 3];

julia> Y = zero(X);

julia> ldiv!(Y, qr(A), X);

julia> Y
3-element Array{Float64,1}:
  0.7128099173553719
 -0.051652892561983674
  0.10020661157024757

julia> A\X
3-element Array{Float64,1}:
  0.7128099173553719
 -0.05165289256198333
  0.10020661157024785
```

---

```
ldiv!(A, B)
```

Compute `A \ B` in-place and overwriting `B` to store the result.

The argument `A` should *not* be a matrix.  Rather, instead of matrices it should be a factorization object (e.g. produced by [`factorize`](@ref) or [`cholesky`](@ref)). The reason for this is that factorization itself is both expensive and typically allocates memory (although it can also be done in-place via, e.g., [`lu!`](@ref)), and performance-critical situations requiring `ldiv!` usually also require fine-grained control over the factorization of `A`.

# Examples

```jldoctest
julia> A = [1 2.2 4; 3.1 0.2 3; 4 1 2];

julia> X = [1; 2.5; 3];

julia> Y = copy(X);

julia> ldiv!(qr(A), X);

julia> X
3-element Array{Float64,1}:
  0.7128099173553719
 -0.051652892561983674
  0.10020661157024757

julia> A\Y
3-element Array{Float64,1}:
  0.7128099173553719
 -0.05165289256198333
  0.10020661157024785
```

---

```
ldiv!(a::Number, B::AbstractArray)
```

Divide each entry in an array `B` by a scalar `a` overwriting `B` in-place.  Use [`rdiv!`](@ref) to divide scalar from right.

# Examples

```jldoctest
julia> B = [1.0 2.0; 3.0 4.0]
2×2 Array{Float64,2}:
 1.0  2.0
 3.0  4.0

julia> ldiv!(2.0, B)
2×2 Array{Float64,2}:
 0.5  1.0
 1.5  2.0
```

---

Solve Hy = rhs for a non-square Hessenberg matrix. Note that `H` is also modified as is it converted to an upper triangular matrix via Given's rotations


In [57]:
cov(Xout')

80×80 Array{Float64,2}:
  1.02066       0.0519534   -0.000215042  …  -0.0739035    0.0959428
  0.0519534     1.04066      0.0309331        0.00234449  -0.0254844
 -0.000215042   0.0309331    1.05566          0.0114238   -0.00456807
  0.00989968    0.0358303    0.0024257       -0.0197221    0.0518869
  0.00748843    0.0347069   -0.0244349        0.00194468   0.0884858
  0.0651829     0.0985913   -0.0413188    …  -0.0241476    0.0240716
 -0.00720177   -0.0280557   -0.0164677       -0.0234285   -0.0481258
  0.0806543    -0.0707258   -0.00281204      -0.0311662   -0.00421105
 -0.0764236    -0.0128012   -0.0689943        0.0126519    0.00823787
  0.0548447    -0.0166302   -0.0294575       -0.00510976  -0.0135642
  0.0362238     0.0544405   -0.11035      …   0.0410722    0.0280195
 -0.051936     -0.106704    -0.0916128        0.0107262    0.00901518
  0.00568182   -0.0149839   -0.127733        -0.0828839   -0.0763533
  ⋮                                       ⋱               
 -0.0990927    -

In [29]:
mean(X; dims = 2)[:,1]

80-element Array{Float64,1}:
  6.66133814775094e-18
  4.274358644806853e-17
  6.29126380620922e-17
  1.313763912473102e-16
  5.736152293896643e-18
  3.090120751873353e-17
  2.275957200481571e-17
  4.070817756958907e-18
  2.960594732333751e-18
  3.7007434154171884e-18
 -2.1649348980190553e-17
 -9.251858538542972e-18
  3.7747582837255325e-17
  ⋮
  1.9354888062631896e-16
 -3.959795454496392e-17
 -2.960594732333751e-18
  2.960594732333751e-18
 -1.2582527612418442e-17
  4.070817756958907e-18
  7.308968245448948e-18
 -3.034609600642095e-17
  3.552713678800501e-17
 -5.714612810736597e-17
  3.7007434154171887e-19
 -7.401486830834377e-18